# Optimizing Performance (Single-Core)

## SIMD

SIMD stands for **"Single Instruction Multiple Data"** and falls into the category of instruction level **parallelism** (vector instructions). Since raw clock speeds haven't been getting much faster, one way in which processors have been able to increase performance is through operations which operate on a "vector" (basically, a short sequence of values contiguous in memory).

Consider this simple vector addition example:

In [1]:
function vector_add(A, B, C)    
    for i in eachindex(A, B, C)
        @inbounds A[i] = B[i] + C[i]
    end
end

vector_add (generic function with 1 method)

<br>

*Packed* vector addition: **vaddpd**

<img src="./imgs/SIMD.svg" width=450px>
<br>


The idea behind SIMD is to perform the add instruction on multiple elements at the same time (instead of separately performing them one after another). The process of splitting up the simple loop addition into multiple vector additions is often denoted as "loop vectorization". Since each vectorized addition happens at instruction level, i.e. within a CPU core, the feature set of the CPU determines how many elements we can process in one go.

#### SIMD register width

<br>
<img src="./imgs/SIMD_vectorwidth.svg" width=580px>
<br>

### Is SIMD important?

**Peak performance** (single-core): $P_\textrm{core} = f \cdot n_\textrm{super} \cdot n_\textrm{FMA} \cdot n_\textrm{SIMD}$
- $f$: clock frequency
- $n_\textrm{super}$: superscalarity (multiple arithmetic units)
- $n_\textrm{FMA}$: FMA factor (two FLOPs in one instruction)
- $n_\textrm{SIMD}$: SIMD factor

|microarchitecture|processor|launch date|f [GHz]|n_super|n_FMA|n_SIMD|P_core [GFLOPS]|
|:----|:----|:----|:----|:----|:----|:----|:----|
|Haswell|Xeon E5-2695 v3|Q3/2014|2.30|2|2|4|36.8|
|Skylake SP|Xeon Gold 6148|Q3/2017|2.40|2|2|8|76.8|
|Zen 2|EPYC 7642|Q3/2019|2.30|2|2|4|36.8|
|Zen 3|EPYC 7763|Q1/2021|2.45|2|2|4|39.2|
|A64FX|FX1000|Q1/2020|2.20|2|2|8|70.4|


### What does my system support?

Let's check which "advanced vector extensions" (AVX) the system supports.

In [2]:
using CpuId
cpuinfo()

| Cpu Property       | Value                                                      |
|:------------------ |:---------------------------------------------------------- |
| Brand              | Intel(R) Core(TM) i9-10900 CPU @ 2.80GHz                   |
| Vendor             | :Intel                                                     |
| Architecture       | :UnknownIntel                                              |
| Model              | Family: 0x06, Model: 0xa5, Stepping: 0x05, Type: 0x00      |
| Cores              | 10 physical cores, 20 logical cores (on executing CPU)     |
|                    | Hyperthreading hardware capability detected                |
| Clock Frequencies  | 2800 / 5200 MHz (base/max), 100 MHz bus                    |
| Data Cache         | Level 1:3 : (32, 256, 20480) kbytes                        |
|                    | 64 byte cache line size                                    |
| Address Size       | 48 bits virtual, 39 bits physical                          |
| SIMD               | 256 bit = 32 byte max. SIMD vector size                    |
| Time Stamp Counter | TSC is accessible via `rdtsc`                              |
|                    | TSC runs at constant rate (invariant from clock frequency) |
| Perf. Monitoring   | Performance Monitoring Counters (PMC) revision 4           |
|                    | Available hardware counters per logical core:              |
|                    | 3 fixed-function counters of 48 bit width                  |
|                    | 4 general-purpose counters of 48 bit width                 |
| Hypervisor         | No                                                         |


In [3]:
filter(x -> contains(string(x), "AVX"), cpufeatures())

2-element Vector{Symbol}:
 :AVX
 :AVX2

**Noctua 2 nodes do not have AVX512.**

In [4]:
SIZE = 512^2
A = rand(Float64, SIZE);
B = rand(Float64, SIZE);
C = rand(Float64, SIZE);

In [5]:
@code_native debuginfo=:none syntax=:intel vector_add(A,B,C)

	.text
	.file	"vector_add"
	.globl	japi1_vector_add_1565           # -- Begin function japi1_vector_add_1565
	.p2align	4, 0x90
	.type	japi1_vector_add_1565,@function
japi1_vector_add_1565:                  # @japi1_vector_add_1565
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 128
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	mov	qword ptr [rsp + 32], 0
	mov	qword ptr [rsp + 96], rsi
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r12, qword ptr [rax - 8]
	mov	qword ptr [rsp], 12
	mov	rax, qword ptr [r12]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r12], rax
	mov	rdx, qword ptr [rsi]
	mov	rax, qword ptr [rsi + 8]
	mov	rcx, qword ptr [rsi + 16]
	mov	r15, qword ptr [rdx + 8]
	mov	rbx, qword ptr [rax + 8]
	mov	r13, qword ptr [rcx + 8]
	cmp	r15, rbx
	jne	.LBB0_20
# %bb.1:                                # %top
	cmp	r15, r13
	jne	.LBB0_20
# %bb.2:                     

 [rcx + 8*rdi + 64]
	vaddpd	ymm3, ymm3, ymmword ptr [rcx + 8*rdi + 96]
	vmovupd	ymmword ptr [rdx + 8*rdi], ymm0
	vmovupd	ymmword ptr [rdx + 8*rdi + 32], ymm1
	vmovupd	ymmword ptr [rdx + 8*rdi + 64], ymm2
	vmovupd	ymmword ptr [rdx + 8*rdi + 96], ymm3
.LBB0_12:                               # %middle.block
	cmp	r15, rsi
	je	.LBB0_19
# %bb.13:
	or	rsi, 1
.LBB0_14:                               # %scalar.ph
	mov	ebx, r15d
	sub	ebx, esi
	inc	ebx
	mov	rdi, r15
	sub	rdi, rsi
	and	rbx, 3
	je	.LBB0_16
	.p2align	4, 0x90
.LBB0_15:                               # %L40.prol
                                        # =>This Inner Loop Header: Depth=1
	vmovsd	xmm0, qword ptr [rax + 8*rsi - 8] # xmm0 = mem[0],zero
	vaddsd	xmm0, xmm0, qword ptr [rcx + 8*rsi - 8]
	vmovsd	qword ptr [rdx + 8*rsi - 8], xmm0
	inc	rsi
	dec	rbx
	jne	.LBB0_15
.LBB0_16:                               # %L40.prol.loopexit
	cmp	rdi, 3
	jb	.LBB0_19
# %bb.17:                               # %L40.preheader82
	sub	r15, rsi
	lea	rax, [r

### It's not always so simple: SIMD can be hard...

Autovectorization is a hard problem (it needs to prove a lot of things about the code!). After all, it is a from of parallelism and efficient parallelism can be hard as well...

Not every loop is (readily) vectorizable. **Keep your loops as simple as possible!**

* avoid conditionals and function calls etc.
* ideally, loop length is static (countable up front).
* access **contiguous data** (spatial locality).
  * (align data structures to SIMD width boundary)
* avoid data dependencies (e.g. between loop iterations)

#### Example: Reduction

In [6]:
function vector_dot(B, C)
    a = zero(eltype(B))
    for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot (generic function with 1 method)

In [7]:
@code_native debuginfo=:none syntax=:intel vector_dot(B, C)

	.text
	.file	"vector_dot"
	.globl	julia_vector_dot_1594           # -- Begin function julia_vector_dot_1594
	.p2align	4, 0x90
	.type	julia_vector_dot_1594,@function
julia_vector_dot_1594:                  # @julia_vector_dot_1594
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r15, qword ptr [rax - 8]
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [r15]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r15], rax
	mov	rbx, qword ptr [rdi + 8]
	mov	r12, qword ptr [rsi + 8]
	cmp	rbx, r12
	jne	.LBB0_12
# %bb.1:                                # %L22
	test	rbx, rbx
	je	.LBB0_2
# %bb.3:                                # %L31.preheader
	mov	rcx, qword ptr [rdi]
	mov	rdx, qword ptr [rsi]
	lea	rsi, [rbx - 1]
	mov	eax, ebx
	and	eax, 7
	cmp	rsi, 7
	jae	.LBB0_5
# %bb.4:
	vxorpd	xmm0, xmm0, 

qword ptr [rsp + 48], rcx
	mov	qword ptr [rsp + 56], r14
	mov	qword ptr [rsp + 64], rax
	movabs	rax, offset ijl_invoke
	movabs	rdi, 140634259335520
	lea	rsi, [rsp + 48]
	movabs	rcx, 140634400698096
	mov	edx, 3
	call	rax
	ud2
.Lfunc_end0:
	.size	julia_vector_dot_1594, .Lfunc_end0-julia_vector_dot_1594
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


Note the `vaddsd` instruction and usage of `xmmi` registers (128 bit).

#### How could this loop reduction be vectorized manually?

In [8]:
function vector_dot_unrolled4(B, C)
    a1 = zero(eltype(B))
    a2 = zero(eltype(B))
    a3 = zero(eltype(B))
    a4 = zero(eltype(B))
    @inbounds for i in 1:4:length(B)-4
        a1 += B[i] * C[i]
        a2 += B[i+1] * C[i+1]
        a3 += B[i+2] * C[i+2]
        a4 += B[i+3] * C[i+3]
    end
    return a1+a2+a3+a4
end

vector_dot_unrolled4 (generic function with 1 method)

In [9]:
@code_native debuginfo=:none syntax=:intel vector_dot_unrolled4(B, C)

	.text
	.file	"vector_dot_unrolled4"
	.globl	julia_vector_dot_unrolled4_1597 # -- Begin function julia_vector_dot_unrolled4_1597
	.p2align	4, 0x90
	.type	julia_vector_dot_unrolled4_1597,@function
julia_vector_dot_unrolled4_1597:        # @julia_vector_dot_unrolled4_1597
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	mov	rdx, qword ptr [rdi + 8]
	lea	rcx, [rdx - 4]
	mov	eax, 1
	cmp	rcx, 1
	je	.LBB0_3
# %bb.1:                                # %L10
	vxorpd	xmm0, xmm0, xmm0
	cmp	rdx, 6
	jb	.LBB0_5
# %bb.2:                                # %L45
	dec	edx
	and	edx, 3
	sub	rcx, rdx
	mov	rax, rcx
	test	rcx, rcx
	jle	.LBB0_5
.LBB0_3:                                # %L59.preheader
	mov	rcx, qword ptr [rdi]
	mov	rdx, qword ptr [rsi]
	add	rax, 3
	vxorpd	xmm0, xmm0, xmm0
	xor	esi, esi
	.p2align	4, 0x90
.LBB0_4:                                # %L59
                                        # =>This Inner Loop Header: Depth=1
	vmovupd	ymm1, ymmword ptr [rcx + 8*rsi]
	vmulpd	ym

In [10]:
using BenchmarkTools
@btime vector_dot($B, $C) samples=10 evals=3;
@btime vector_dot_unrolled4($B, $C) samples=10 evals=3;

  214.549 μs (0 allocations: 0 bytes)


  54.851 μs (0 allocations: 0 bytes)


#### The "automatic" way: `@simd`

To (try to) "force" automatic SIMD vectorization in Julia, you can use the `@simd` macro.

In [11]:
function vector_dot_simd(B, C)
    a = zero(eltype(B))
    @simd for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot_simd (generic function with 1 method)

By using the `@simd` macro, we are **asserting several properties** of the loop:

* It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
* Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In [12]:
@btime vector_dot_simd($B, $C) samples=10 evals=3;

  50.915 μs (0 allocations: 0 bytes)


In [13]:
@code_native debuginfo=:none syntax=:intel vector_dot_simd(B, C)

	.text
	.file	"vector_dot_simd"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_vector_dot_simd_1766
.LCPI0_0:
	.quad	0x8000000000000000              # double -0
	.section	.rodata.cst32,"aM",@progbits,32
	.p2align	5
.LCPI0_1:
	.quad	0x0000000000000000              # double 0
	.quad	0x8000000000000000              # double -0
	.quad	0x8000000000000000              # double -0
	.quad	0x8000000000000000              # double -0
	.text
	.globl	julia_vector_dot_simd_1766
	.p2align	4, 0x90
	.type	julia_vector_dot_simd_1766,@function
julia_vector_dot_simd_1766:             # @julia_vector_dot_simd_1766
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r15, qword ptr [rax - 8]
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [r15]
	mov	qwor

xmm1, qword ptr [rax + 8*rdx]   # xmm1 = mem[0],zero
	vfmadd231sd	xmm0, xmm1, qword ptr [rcx + 8*rdx] # xmm0 = (xmm1 * mem) + xmm0
	inc	rdx
	cmp	rbx, rdx
	jne	.LBB0_12
.LBB0_13:                               # %L50
	mov	rax, qword ptr [rsp + 8]
	mov	qword ptr [r15], rax
	lea	rsp, [rbp - 40]
	pop	rbx
	pop	r12
	pop	r13
	pop	r14
	pop	r15
	pop	rbp
	vzeroupper
	ret
.LBB0_6:
	xor	esi, esi
	vmovapd	ymm2, ymm0
	vmovapd	ymm3, ymm0
	test	r8b, 1
	je	.LBB0_11
	jmp	.LBB0_10
.LBB0_14:                               # %L12
	mov	rdi, qword ptr [r15 + 16]
	movabs	r14, offset ijl_gc_pool_alloc
	mov	esi, 752
	mov	edx, 16
	vzeroupper
	call	r14
	mov	rcx, r14
	mov	r14, rax
	movabs	r13, 140634244783472
	mov	qword ptr [rax - 8], r13
	mov	qword ptr [rax], rbx
	mov	qword ptr [rsp + 24], rax
	mov	rdi, qword ptr [r15 + 16]
	mov	esi, 752
	mov	edx, 16
	call	rcx
	mov	qword ptr [rax - 8], r13
	mov	qword ptr [rax], r12
	mov	qword ptr [rsp + 16], rax
	movabs	rcx, 140634189291360
	mov	qword ptr [rsp + 48], rcx
	mov	qword

Note the `vfmadd231pd` instruction and usage of `ymmi` AVX registers (256 bit).

#### Data types matter
Floating-point addition is **non-associative** and the order of operations is important.

In [14]:
v = rand(10^6)
@show vector_dot(v,v);
@show vector_dot_simd(v,v);
@show abs(vector_dot(v,v) - vector_dot_simd(v,v));

vector_dot(v, v) = 333295.5488100384
vector_dot_simd(v, v) = 333295.54881005303
abs(vector_dot(v, v) - vector_dot_simd(v, v)) = 1.461012288928032e-8


How bad can this get? In principle, [arbitraily bad](https://discourse.julialang.org/t/when-shouldnt-we-use-simd/18276/11?u=carstenbauer)!! Quite often you can get away with it though.


Compare this to integer addition, which is **associative** and the order of operations has no impact.

In [15]:
B_int = rand(Int64, SIZE);
C_int = rand(Int64, SIZE);

In [16]:
@show vector_dot(B_int, C_int);
@show vector_dot_simd(B_int, C_int);
@show abs(vector_dot(B_int, C_int) - vector_dot_simd(B_int, C_int));

vector_dot(B_int, C_int) = 6972760643415222650
vector_dot_simd(B_int, C_int) = 6972760643415222650
abs(vector_dot(B_int, C_int) - vector_dot_simd(B_int, C_int)) = 0


In [17]:
@btime vector_dot($B_int, $C_int) samples=10 evals=3;;
@btime vector_dot_simd($B_int, $C_int) samples=10 evals=3;;

  80.755 μs (0 allocations: 0 bytes)


  78.111 μs (0 allocations: 0 bytes)


#### Data layout matters (Structure of Array vs Array of Structure)

Contiguous memory access facilitates SIMD.

In [18]:
complex_numbers_aos = [rand() + im * rand() for i in 1:1024] # array of structs (Complex{Float64})

1024-element Vector{ComplexF64}:
 0.11140534006308322 + 0.2117217096992412im
 0.30938696589108483 + 0.0748780748049993im
  0.6422909659691299 + 0.30068890090083944im
 0.29564005706093344 + 0.6742292912099301im
  0.4897448308858604 + 0.9548675633236571im
 0.23688309081509484 + 0.142747253462708im
 0.15979570838326052 + 0.29548286055060513im
  0.8774669943012473 + 0.5520512468201935im
  0.9829291284167867 + 0.40167383589890404im
  0.9271092995252095 + 0.8826011541564129im
                     ⋮
  0.5287117721087015 + 0.4676385108950667im
 0.08657345220603652 + 0.9022028743819831im
  0.3245273103462415 + 0.7390409452748052im
  0.1457070806937092 + 0.03450288061413376im
 0.41427382010020175 + 0.6965868278456187im
  0.2899756377868904 + 0.9257746692814004im
  0.8091186941205888 + 0.3790156423778198im
  0.9278030438421158 + 0.7703536046236622im
  0.5927690443129342 + 0.46317706294595296im

In [19]:
import Base: sum

struct ComplexNumbers
    x::Vector{Float64}
    y::Vector{Float64}
end

sum(cn::ComplexNumbers) = sum(cn.x) + im * sum(cn.y)

sum (generic function with 11 methods)

In [20]:
complex_numbers_soa = ComplexNumbers(rand(1024), rand(1024)) # struct of arrays

ComplexNumbers([0.40981738410809865, 0.030086783716106402, 0.8577860104636237, 0.43089071053719563, 0.5891308806782146, 0.7521876858909056, 0.5385380858681411, 0.3918709850980979, 0.02421479953543515, 0.47146559665491095  …  0.8299829904371342, 0.7579691071806107, 0.39184799641502244, 0.6798736537882101, 0.3346541021242325, 0.0746285587159542, 0.2535157962829193, 0.1423187741907439, 0.5827030592633646, 0.7289579190777635], [0.6407370531331318, 0.9988233883160311, 0.02940393218703463, 0.44850225247020836, 0.5468038775060854, 0.6291537043475894, 0.17464489725564947, 0.7246074608249262, 0.46873656388468077, 0.7229737793055284  …  0.49850145590399364, 0.5255432532747653, 0.3086111814694149, 0.7575844784990997, 0.19737495845458797, 0.612135437057205, 0.5536071499180154, 0.9437221802582934, 0.5409704536934015, 0.4682191010623693])

In [21]:
@btime sum($complex_numbers_aos);
@btime sum($complex_numbers_soa);

  788.750 ns (0 allocations: 0 bytes)


  130.845 ns (0 allocations: 0 bytes)


Sidenote: [StructArrays.jl](https://github.com/JuliaArrays/StructArrays.jl)

## Other tricks: `@fastmath` (if time permits)

Enables lots of floating point optimizations that are potentially *unsafe*! It trades accuracy for speed, so, [Beware of fast-math](https://simonbyrne.github.io/notes/fastmath/). (See the [LLVM Language Reference Manual](https://llvm.org/docs/LangRef.html#fast-math-flags) for more information on which compiler options it sets.)

### SIMD
Among other things, it **facilitates SIMD vectorization** because it:
* Allows re-association of operands in series of floating-point operations.

In [22]:
function vector_dot_fastmath(B, C)
    a = zero(eltype(B))
    @fastmath for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot_fastmath (generic function with 1 method)

In [23]:
@btime vector_dot_fastmath($B, $C) samples=10 evals=3;

  64.473 μs (0 allocations: 0 bytes)


In [24]:
@code_native debuginfo=:none syntax=:intel vector_dot_fastmath(B,C)

	.text
	.file	"vector_dot_fastmath"
	.globl	julia_vector_dot_fastmath_2094  # -- Begin function julia_vector_dot_fastmath_2094
	.p2align	4, 0x90
	.type	julia_vector_dot_fastmath_2094,@function
julia_vector_dot_fastmath_2094:         # @julia_vector_dot_fastmath_2094
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	r15, qword ptr [rax - 8]
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [r15]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r15], rax
	mov	rbx, qword ptr [rdi + 8]
	mov	r12, qword ptr [rsi + 8]
	cmp	rbx, r12
	jne	.LBB0_16
# %bb.1:                                # %L22
	test	rbx, rbx
	je	.LBB0_2
# %bb.3:                                # %L31.preheader
	mov	rax, qword ptr [rdi]
	mov	rcx, qword ptr [rsi]
	cmp	rbx, 16
	jae	.LBB0_5
# %bb.4:
	vxorpd	xmm0, xmm0, xmm0
	mov

 ptr [r15], rax
	lea	rsp, [rbp - 40]
	pop	rbx
	pop	r12
	pop	r13
	pop	r14
	pop	r15
	pop	rbp
	vzeroupper
	ret
.LBB0_6:
	vxorpd	xmm0, xmm0, xmm0
	xor	esi, esi
	vxorpd	xmm1, xmm1, xmm1
	vxorpd	xmm2, xmm2, xmm2
	vxorpd	xmm3, xmm3, xmm3
	test	r8b, 1
	je	.LBB0_11
	jmp	.LBB0_10
.LBB0_16:                               # %L11
	mov	rdi, qword ptr [r15 + 16]
	movabs	r14, offset ijl_gc_pool_alloc
	mov	esi, 752
	mov	edx, 16
	vzeroupper
	call	r14
	mov	rcx, r14
	mov	r14, rax
	movabs	r13, 140634244783472
	mov	qword ptr [rax - 8], r13
	mov	qword ptr [rax], rbx
	mov	qword ptr [rsp + 24], rax
	mov	rdi, qword ptr [r15 + 16]
	mov	esi, 752
	mov	edx, 16
	call	rcx
	mov	qword ptr [rax - 8], r13
	mov	qword ptr [rax], r12
	mov	qword ptr [rsp + 16], rax
	movabs	rcx, 140634189291360
	mov	qword ptr [rsp + 48], rcx
	mov	qword ptr [rsp + 56], r14
	mov	qword ptr [rsp + 64], rax
	movabs	rax, offset ijl_invoke
	movabs	rdi, 140634259335520
	lea	rsi, [rsp + 48]
	movabs	rcx, 140634400698096
	mov	edx, 3
	call	rax
	ud2
.Lfunc

### FMA - Fused Multiply Add

In [25]:
f(a,b,c) = a*b+c

f (generic function with 1 method)

In [26]:
@code_native debuginfo=:none f(1.0,2.0,3.0)

	.text
	.file	"f"
	.globl	julia_f_2103                    # -- Begin function julia_f_2103
	.p2align	4, 0x90
	.type	julia_f_2103,@function
julia_f_2103:                           # @julia_f_2103
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	vmulsd	xmm0, xmm0, xmm1
	vaddsd	xmm0, xmm0, xmm2
	pop	rbp
	ret
.Lfunc_end0:
	.size	julia_f_2103, .Lfunc_end0-julia_f_2103
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


In [27]:
f_fastmath(a,b,c) = @fastmath a*b+c

f_fastmath (generic function with 1 method)

In [28]:
@code_native debuginfo=:none f_fastmath(1.0,2.0,3.0)

	.text
	.file	"f_fastmath"
	.globl	julia_f_fastmath_2106           # -- Begin function julia_f_fastmath_2106
	.p2align	4, 0x90
	.type	julia_f_fastmath_2106,@function
julia_f_fastmath_2106:                  # @julia_f_fastmath_2106
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	vfmadd213sd	xmm0, xmm1, xmm2        # xmm0 = (xmm1 * xmm0) + xmm2
	pop	rbp
	ret
.Lfunc_end0:
	.size	julia_f_fastmath_2106, .Lfunc_end0-julia_f_fastmath_2106
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


(In this specific case, the explicit `fma` function or [MuladdMacro.jl](https://github.com/SciML/MuladdMacro.jl) are *safer* alternatives.)

<img src="./imgs/skylake_server_microarch.png" width=900px>

**Source:** [Intel® 64 and IA-32 Architectures Optimization Reference Manual](https://software.intel.com/sites/default/files/managed/9e/bc/64-ia-32-architectures-optimization-manual.pdf)

#### Sidenote: Why doesn't Julia use FMA automatically?

Answer: because it can break math in weird ways.

In [29]:
function f(a,b,c)
    @assert a*b ≥ c
    return sqrt(a*b-c)
end

function f_fma(a,b,c)
    @assert a*b ≥ c
    return sqrt(fma(a,b,-c))
end

a = 1.0 + 0.5^27;
b = 1.0 - 0.5^27;
c = 1.0;

In [30]:
f(a,b,c)

0.0

In [31]:
f_fma(a,b,c)

DomainError: DomainError with -5.551115123125783e-17:
sqrt was called with a negative real argument but will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

# Core messages of this Notebook

* **SIMD is important for your innermost computational kernel** and, ideally, can give you a factor of 4 or 8 speedup (for `Float64`).
* **Keep your hot loop as simple as possible** to facilitate SIMD (avoid branches, data dependencies, etc., if possible).
* (Carefully) think about using `@simd`, `@fastmath`, etc. to **opt-into potentially unsafe optimizations**.